1. 웹사이트 레이아웃: 지정한 레이아웃에 맞게 다 긁어온다. 해당 페이지에서.

2. 링크 : 
    - 검색: 지정한 페이지에서 검색어 쿼리 붙여서 해당하는 거 다 들고 온다.
    - 링크 : 지정한 페이지에서 해당 링크로 시작하는 거 다 긁어온다! 로이터 예제의 경우에는 첫 화면에 있는 모든 링크들. article로 시작하는!

3. 

# 1. 웹사이트 레이아웃에 따른 크롤러

---

* 웹사이트 구조 파악해야 한다.
    - 예컨대, NYTimes의 경우에는 글을 적는 `StoryBodyCompanionColumn` 아래의 p 태그가 여러 개로 흩어져 존재했기 때문에, `bs.select`를 사용해서 전체 다 찾아줘야 한다. 만약 find를 사용하고 싶었다면, find가 아니라 `find_all`을 사용해야 했을 것. 
    - 반면, brookings의 경우에는 본문을 나타내는 태그가 하나밖에 없었으므로 find를 통해 하나의 태그만 찾아줘도 된다.

* "유연성이 떨어진다"는 단점이 있다.
    - `ver1` 함수의 경우, 각 웹사이트에서 htmL을 선택하고 구문 분석을 할 수 있는 전용 함수를 서서 결과를 얻어야 한다.
    - `ver2` 함수의 경우, 대상 필드가 반드시 존재해야 하고, 데이터를 필드에서 깨끗하게 추출할 수 있어야 하며, 각 대상 필드에 고유하고 신뢰할 수 있는 CSS 선택자가 존재해야 한다.

## ver1

1. 각 사이트에 대한 페이지 구문 분석기.

2. 사이트별로 정보를 얻는 데 사용되는 CSS 선택자만 달라짐.

3. 작동 원리
    * `Content` 클래스
        - 웹 사이트 컨텐츠에 대응.
  
    * 스크레이퍼 함수 : `Content` 클래스로부터 `Content` 인스턴스 반환.
        - 제목 요소 선택, 제목 텍스트 추출.
        - 기사 주요 컨텐츠 선택.
        - (필요하면) 다른 컨텐츠 선택.
        - 위의 문자열들로 인스턴스화된 `Content` 객체 반환.

---
Q) CSS에서 class 선택 시 모든 이름을 다 적지 않아도 되는 건가?
* NYT 스크레이핑에는 select를,
* brookings 스크레이핑에는 find 사용했다.

In [23]:
import requests
from bs4 import BeautifulSoup

# Content 클래스 생성 : 사이트의 url, 제목, 본문을 인스턴스로 반환.
class Content:
    def __init__(self, url, title, body):
        self.url = url
        self.title = title
        self.body = body

# requests : GET 요청을 보내 페이지를 읽어 옴.
def getPage(url):
    req = requests.get(url)
    return BeautifulSoup(req.text, 'html.parser')

# NYTimes 페이지를 스크레이핑 한 뒤, content 인스턴스로 반환.
def scrapeNYTimes(url):
    bs = getPage(url) # getPage 함수를 통해 BeautifulSoup 객체 반환.
    title = bs.find("h1").text # 가장 처음 h1 태그의 text 반환.
    lines = bs.select("div.StoryBodyCompanionColumn div p")
    body = "\n".join([line.text for line in lines])
    return Content(url, title, body)

# Brookings 페이지를 스크레이핑하고, content 인스턴스로 반환.
def scrapingBrookings(url):
    bs = getPage(url)
    title = bs.find("h1").text
    body = bs.find("div", {"class" : "post-body"}).text    
    return Content(url, title, body)

# NYTimes 홈페이지 스크레이핑
url_NYT = "https://www.nytimes.com/2020/03/22/health/coronavirus-restrictions-us.html"
NYT = scrapeNYTimes(url_NYT)
print("NYT 스크레이핑 결과")
print("제목 : {}".format(NYT.title))
print("URL : {}".format(NYT.url))
print("본문 : {}".format(NYT.body))

# Brookings 홈페이지 스크레이핑
url_Brookings ="https://www.brookings.edu/events/bloggers-buzz-and-soundbites-innovative-media-approaches-to-humanitarian-response/"
Brookings = scrapingBrookings(url_Brookings)
print("Brookings 스크레이핑 결과")
print("제목 : {}".format(Brookings.title))
print("URL : {}".format(Brookings.url))
print("본문 : {}".format(Brookings.body))

NYT 스크레이핑 결과
제목 : The Virus Can Be Stopped, but Only With Harsh Steps, Experts Say
URL : https://www.nytimes.com/2020/03/22/health/coronavirus-restrictions-us.html
본문 : Terrifying though the coronavirus may be, it can be turned back. China, South Korea, Singapore and Taiwan have demonstrated that, with furious efforts, the contagion can be brought to heel.
Whether they can keep it suppressed remains to be seen. But for the United States to repeat their successes will take extraordinary levels of coordination and money from the country’s leaders, and extraordinary levels of trust and cooperation from citizens. It will also require international partnerships in an interconnected world.
There is a chance to stop the coronavirus. This contagion has a weakness.
Although there are incidents of rampant spread, as happened on the cruise ship Diamond Princess, the coronavirus more often infects clusters of family members, friends and work colleagues, said Dr. David L. Heymann, who chairs an exp

Brookings 스크레이핑 결과
제목 : Bloggers, Buzz and Soundbites: Innovative Media Approaches to Humanitarian Response
URL : https://www.brookings.edu/events/bloggers-buzz-and-soundbites-innovative-media-approaches-to-humanitarian-response/
본문 : 
The internet is playing an increasingly important role in mobilizing humanitarian response. Grassroots coalitions are now built by e-mail and text messages. Mainstream media outlets continually rely on stories researched and broken by bloggers and are no putting more resources into their own websites. Bloggers and independent journalists are even utilizing other internet tools, such as YouTube, in addition to their blogs as a way of raising awareness about issues as well as for fundraising. Such wide access to the internet and the proliferation of blogs has enabled coverage of war and humanitarian issues from different perspectives.
This seminar explored the role of new media technologies in mobilizing humanitarian response. What is the balance between m

In [21]:
# NYTimes 홈페이지 스크레이핑 테스트 : 궁금증 해소용.
def scrapeNYTimes_test(url):
    bs = getPage(url) 
    title = bs.find("h1") 
    body = bs.select("div.css-1fanzo5.StoryBodyCompanionColumn div p") # 이렇게 해도 되고
    return Content(url, title, body)

url_NYT = "https://www.nytimes.com/2020/03/22/health/coronavirus-restrictions-us.html"
content = scrapeNYTimes_test(url_NYT)
print("Title : {}".format(content.title))
print("URL : {}".format(content.url))
print(content.body[0:2]) # 교재와 달리 text 뽑아오지 않고, line에 개행문자 join해주지 않으면 가독성이 떨어진다.

Title : <h1 class="css-1rtkyej e1h9rw200" id="link-41c85874" itemprop="headline"><span>The Virus Can Be Stopped, but Only With Harsh Steps, Experts Say</span></h1>
URL : https://www.nytimes.com/2020/03/22/health/coronavirus-restrictions-us.html
[<p class="css-exrw3m evys1bk0">Terrifying though the coronavirus may be, it can be turned back. China, South Korea, Singapore and Taiwan have demonstrated that, with furious efforts, the contagion can be brought to heel.</p>, <p class="css-exrw3m evys1bk0">Whether they can keep it suppressed remains to be seen. But for the United States to repeat their successes will take extraordinary levels of coordination and money from the country’s leaders, and extraordinary levels of trust and cooperation from citizens. It will also require international partnerships in an interconnected world.</p>]


### 네이버 뉴스 홈페이지에서 실습

* 주석까지 다 나오는데?
* 마지막 세 줄은 어떻게 뺄 수 없을까?

In [26]:
class Content:
    def __init__(self, url, title, body):
        self.url = url
        self.title = title
        self.body = body

def getPage(url):
    req = requests.get(url)
    return BeautifulSoup(req.text, 'html.parser')

def scrapeNaverNews(url):
    bs = getPage(url)
    title = bs.find("h3").text
    body = bs.find("div", {"id" : "articleBodyContents"}).text
    return Content(url, title, body)

naver_news_url = "https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=079&aid=0003339552"
naver_news = scrapeNaverNews(naver_news_url)

print("네이버 뉴스 스크레이핑 결과")
print("제목 : {}".format(naver_news.title))
print("URL : {}".format(naver_news.url))
print("본문 : {}".format(naver_news.body))
    

네이버 뉴스 스크레이핑 결과
제목 : 헤드라인 뉴스
URL : https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=079&aid=0003339552
본문 : 




// flash 오류를 우회하기 위한 함수 추가
function _flash_removeCallback() {}

생활가전·스마트폰·자동차 생산라인 임시 가동 중단글로벌 공급망 붕괴 우려에 인도 거점 생산·판매 전략도 비상[CBS노컷뉴스 최인수·송영훈 기자](사진=연합뉴스)코로나19 확산으로 유럽과 미국은 물론 인도까지 한국 기업들의 생활가전·스마트폰·자동차 생산라인이 잇달아 셧다운됐다. 글로벌 공급망 붕괴 우려와 인도를 거점으로 삼아 생산과 판매를 늘려오던 기업들로서는 비상이 걸린 셈이다.23일 업계에 따르면, 인도 정부의 코로나19 대응에 따라 삼성전자와 LG전자, 현대차 현지 공장이 임시 가동 중단됐다.삼성전자는 인도 주정부 지침에 따라 노이다 공장을 오는 25일가지 가동 중단한다. 노이다 공장은 삼성전자의 스마트폰을 주로 생산하고 있다. 노이다 법인은 일부 가전도 생산한다.LG전자도 노이다와 푸네에 위치한 생산법인을 3월 말까지 중단하기로 했다. LG전자 노이다 공장과 푸네 공장에서는 세탁기 등 가전제품을 생산하고 있다. 푸네 공장에서는 스마트폰도 일부 생산한다.업계 관계자는 “현지 주정부의 방침에 따라 가동을 중단했다”며 “해당 기간 이후 생산재개를 계획하고 있지만 현지 상황을 예의주시하고 있다”고 말했다.현대차도 인도 첸나이 공장 가동을 이달 말까지 중단한다. 현대차 첸나이공장은 SUV 크레타과 싼타페, 이온, i20, 엘란트라, 엑센트 등의 차종을 연간 68만여대를 생산한다.기아차의 인도 현지 공장인 안드라프라데시 공장은 아직 주정부의 지침이 내려오지 않아 가동 중단에 들어가지 않았지만 임직원의 건강을 고려해 가동 중단을 검토하고 있다. 현대차와 마찬가지로 셧다운 될 가능성이 매우 높은 상황이다.앞서 현대기아차는 미국

## ver2

> `ver1` 함수는 사이트별로 각 정보를 얻는 데 사용한 CSS 선택자를 바꿔주었다. `ver2`에서는 이전 함수의 불편함을 개선하여 CSS 선택자를 문자열로 만들고, 이를 딕셔너리 객체에 모아서 매개변수로 넘긴다.


1. 사이트 구조와 데이터 위치를 정의하는 문자열과 키/값 속성을 딕셔너리로, 매개변수로 넘긴다.
2. 결과 출력 기능을 `Content` 클래스에 넣어서, print 메서드를 호출할 때 원하는 내용만 출력할 수 있도록 바꾼다.
3. 작동원리
    * `Website` 클래스 : 각 페이지에서 특정 데이터를 수집하는 방법을 저장. 예컨대, 제목 텍스트가 아니라, 제목을 찾을 수 있는 위치를 나타내는 문자열 태그 `h1` 자체를 저장한다.
  
    * `Content` 클래스 : 개별 페이지에서 수집한 정보를 저장한다. 결과 출력 기능까지 가지고 있다.
    
    * `Crawler` 클래스 : `Content`와 `Website` 클래스를 사용해 주어진 웹페이지에 존재하는 URL의 제목과 내용을 모두 스크랩한다.
            - getPage: 페이지를 불러온다. 오류는 `RequestException`으로 해결한다.
            - safeGet : 예상되는/확인한 셀렉터 인자로 넘겨서, 문자열 추출.

In [29]:
# module import
import requests
from bs4 import BeautifulSoup

# 글, 홈페이지 전체에 사용할 클래스. 개별 페이지에서 수집한 정보를 저장하고 출력한다.
class Content:
    def __init__(self, url, title, body):
        self.url = url
        self.title = title
        self.body = body
    
    # print
    def print(self):
        print("URL : {}".format(self.url))
        print("제목 : {}".format(self.title))
        print("본문 : {}".format(self.body))

# 웹사이트의 구조 자체에 대한 정보를 저장할 클래스. 제목, 본문 등 어느 태그에 위치한지 저장한다.
class Website:
    def __init__(self, name, url, titleTag, bodyTag):
        self.name = name
        self.url = url
        self.titleTag = titleTag
        self.bodyTag = bodyTag

# 주어진 웹 페이지의 제목과 본문을 스크레이핑하는 클래스.
class Crawler:
    # 페이지 불러오기
    def getPage(self, url):
        try:
            req = requests.get(url)
        except requests.exceptions.RequestException:
            print("접속에 실패했습니다.")
            return None
        return BeautifulSoup(req.text, 'html.parser')
    
    # BeautifulSoup 객체와 selector를 통해 컨텐츠 문자열 추출.
    def safeGet(self, pageObj, selector):
        selectedElems = pageObj.select(selector) # 셀렉터 이용해 선택한 요소들.
        if selectedElems is not None and len(selectedElems) > 0 : 
            return "\n".join([elem.get_text() for elem in selectedElems])
        return "" # 만약 주어진 선택자로 검색된 결과가 없으면 빈 문자열 반환.
    
    # URL을 받아 컨텐츠 추출.
    def parse(self, site, url):
        bs = self.getPage(url) # 컨텐츠 추출할 BeautifulSoup 객체.
        if bs is not None:
            title = self.safeGet(bs, site.titleTag) # safeGet 함수를 이용해 추출한 titletag.
            body = self.safeGet(bs, site.bodyTag)
            if title !="" and body != "": # 제목과 본문이 있다면,
                content = Content(url, title, body) # content 인스턴스로 반환.
                content.print() # print 메서드로 출력.             

In [30]:
# crawler 객체 생성
myCrawler = Crawler()

# 크롤링할 사이트들 : 이름, url, titleTag, bodyTag 순서로 입력해야 함.
sites = [
    ["O\'Reily Media", "http://oreilly.com", "h1", "section#product-description"],
    ['Reuters', "http://reuters.com", 'h1', 'div.StandardArticleBody_body'],
    ['Brookings', 'http://www.brookings.edu', 'h1', 'div.post-body'],
    ['New York Times', 'http://nytimes.com', 'h1', 'div.StoryBodyCompanionColumn div p']
]

websites = []

for site in sites:
    # Website 클래스 구조에 맞게 인스턴스 저장.
    websites.append(Website(site[0], site[1], site[2], site[3]))

oreily_url = "http://shop.oreilly.com/product/0636920085423.do"
reuter_url = "https://www.reuters.com/article/central-banks-deploy-record-sums-to-break-financial-logjam-but-may-need-more-idUSKBN21A0EQ"
brookings_url = "https://www.brookings.edu/events/bloggers-buzz-and-soundbites-innovative-media-approaches-to-humanitarian-response/"
nyt_url = "https://www.nytimes.com/2020/03/22/health/coronavirus-restrictions-us.html"

# 스크레이핑
myCrawler.parse(websites[0], oreily_url)
myCrawler.parse(websites[1], reuter_url)
myCrawler.parse(websites[2], brookings_url)
myCrawler.parse(websites[3], nyt_url)

URL : http://shop.oreilly.com/product/0636920085423.do
제목 : Machine Learning with Python Cookbook
본문 : 
This practical guide provides nearly 200 self-contained recipes to help you solve machine learning challenges you may encounter in your daily work. If you’re comfortable with Python and its libraries, including pandas and scikit-learn, you’ll be able to address specific problems such as loading data, handling text or numerical data, model selection, and dimensionality reduction and many other topics.

Each recipe includes code that you can copy and paste into a toy dataset to ensure that it actually works. From there, you can insert, combine, or adapt the code to help construct your application. Recipes also include a discussion that explains the solution and provides meaningful context. This cookbook takes you beyond theory and concepts by providing the nuts and bolts you need to construct working machine learning applications. 

You’ll find recipes for:

Vectors, matrices, and arra

URL : https://www.brookings.edu/events/bloggers-buzz-and-soundbites-innovative-media-approaches-to-humanitarian-response/
제목 : Bloggers, Buzz and Soundbites: Innovative Media Approaches to Humanitarian Response
Bloggers, Buzz and Soundbites: Innovative Media Approaches to Humanitarian Response
본문 : 
The internet is playing an increasingly important role in mobilizing humanitarian response. Grassroots coalitions are now built by e-mail and text messages. Mainstream media outlets continually rely on stories researched and broken by bloggers and are no putting more resources into their own websites. Bloggers and independent journalists are even utilizing other internet tools, such as YouTube, in addition to their blogs as a way of raising awareness about issues as well as for fundraising. Such wide access to the internet and the proliferation of blogs has enabled coverage of war and humanitarian issues from different perspectives.
This seminar explored the role of new media technologies i

# 2. 웹사이트 링크를 직접 찾고 스크레이핑하는 크롤러 구성

---

 위에서 페이지 레이아웃에 따라 확장성 있는 크롤러를 만들 수 있었다. 그러나 마지막 단계에서와 같이 일일이 url과 링크를 지정해주는 것은 매우 비효율적이다. 따라서, 유연하고 확장성 있는 스크레이핑 기능에 더해 3장에서 살펴 본 웹사이트 링크 수집 및 스크레이핑 기능을 통합한다.
 
 * 검색을 통한 사이트 크롤링.
 * 링크를 통한 사이트 크롤링.
 * 여러 페이지 유형 크롤링

## 2.1. 검색을 통한 사이트 크롤링

 웹사이트에서 키워드나 주제를 검색하고,그 결과 목록을 수집한다. 검색 및 검색 결과 페이지의 URL을 찾아 표준화함으로써 웹사이트 형태를 알고, 페이지에서 데이터를 추출할 수 있다.
 
 * URL에 검색어를 삽입해 검색 결과를 얻을 수 있다. 대부분, `?` 이후 query string이 들어간다.
 * 검색 결과 페이지는 `href` 등 식별이 쉬운 링크 목록으로 제공되는 경우가 많고, `<span class = "result">` 등 사용하기 편리한 태그로 둘러싸여 있다. 이를 위의 `Website` 객체의 속성으로 저장하면 쉽다.
 * 결과 링크가 상대 URL이거나 절대 URL일 수 있다. 어떤 유형의 URL이 필요한지 객체 속성으로 저장할 수 있다.
 
 ---
 위와 같이 검색 페이지의 URL이 표준화될 수 있다면, 웹사이트 형태를 알고 있는 상황에서 데이터를 추출하는 앞의 상황을 그대로 적용하면 된다.
 
1. `Content` 클래스
    * (추가) `URL`: 컨텐츠 위치 추적.

2. `Website` 클래스
    * (추가) `searchUrl` : URL에 검색어 추가했을 때, 검색 결과를 어디에서 얻는지를 저장.
    * (추가) `resultListing` : 각 검색 결과에 대한 정보를 담고 있음.
    * (추가) `resultUrl` : 검색 결과에서 정확한 url을 추출할 때 사용할 태그 정보.
    * (추가) `absoluteUrl` : 검색 결과가 절대 url인지 상대 url인지 알려주는 불리언 값.
    
3. `Crawler` 클래스
    * 이중 루프 : `Website` 데이터, 검색어 목록, 웹사이트 전체에 대해 검색어 전체를 검색.
    * `search` 함수 : 특정 웹사이트 및 주제에 대한 검색 페이지로 이동해서 해당 페이지에 나열된 결과 URL을 모두 추출.

4. 스크랩
    * `topics` 리스트 항목 반복.
    * 스크랩 시작 전 어떤 주제에 대해 스크랩하는지 알림.
    * `sites` 리스트 사이트 반복하며, 외부 루프에서 지정한 검색어로 사이트 스크랩. 
 
 

In [50]:
# module import
import requests
from bs4 import BeautifulSoup

# Content 클래스 : 글, 홈페이지 전체에 대해 적용할 클래스
class Content:
    def __init__(self, topic, url, title, body):
        self.topic = topic
        self.title = title
        self.body = body
        self.url = url
    
    def print(self):
        print("내용을 찾았습니다 : {}".format(self.topic))
        print("URL : {}".format(self.url))
        print("제목 : {}".format(self.title))
        print("본문 : \n {}".format(self.body))

# Website 클래스 : 웹사이트 구조에 대한 정보.
class Website :
    def __init__(self,name, url, searchUrl, resultListing, resultUrl, absoluteUrl, titleTag, bodyTag):
        self.name = name
        self.url = url
        self.searchUrl = searchUrl
        self.resultListing = resultListing
        self.resultUrl = resultUrl
        self.absoluteUrl = absoluteUrl
        self.titleTag = titleTag
        self.bodyTag = bodyTag

# Crawler 클래스 : 웹사이트 URL, 정보 추출.
class Crawler:    
    # 페이지 접속 기능
    def getPage(self, url):
        session = requests.Session()
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
               'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}
        try:
            req = session.get(url, headers = headers)
            print("접속 상태 코드 : {}".format(req.status_code)) # 접속 상태 확인하기 위한 코드.
        except requests.exceptions.RequestException as e:
            print("에러 : {}".format(e))
            return None
        bs = BeautifulSoup(req.text, 'html.parser')
        return bs
    
    # 셀렉터 이용해 태그 선택.    
    def safeGet(self, pageObj, selector):
        childObj = pageObj.select(selector) # CSS 선택자 사용해 선택.
        if childObj is not None and len(childObj) > 0:
            return childObj[0].get_text() # 왜 처음인가??????????
        return ''
    
    # 주어진 검색어(topic)로 특정 웹사이트(site)를 검색해 결과 페이지 모두 기록.
    def search(self, topic, site):
        bs = self.getPage(site.searchUrl + topic) # 검색할 페이지 + 검색 주제(쿼리 스트링).
        searchResults = bs.select(site.resultListing) # 검색 결과 표시되는 영역의 태그로 선택.
        for result in searchResults:
            url = result.select(site.resultUrl)[0].attrs['href'] # 검색 결과 태그 선택하고, 거기서 하이퍼링크 속성 추출.
            # 절대 주소인지 상대 주소인지 체크.
            if (site.absoluteUrl) : # 절대주소라면
                bs = self.getPage(url) # url 그대로 넣어서 페이지 접속
            else: # 상대주소라면
                bs = self.getPage(site.url + url) # 사이트 url 뒤로 url 붙여줘야 함.
            if bs is None: # 오류 나서 접속 안 되면
                print("페이지에 오류가 있습니다. Skipping!")
                return
            title = self.safeGet(bs, site.titleTag) # 제목 태그에 해당하는 요소 가져오기.
            body = self.safeGet(bs, site.bodyTag)
            if title != "" and body != "":
                content = Content(topic, url, title, body)
                content.print()

In [40]:
# Crawler 객체 생성
myCrawler = Crawler()

# 사이트 정보
sites = [
    ["O\'Reilly Media",                      # Website.name
    "http://oreilly.com",                    # Website.url
    "https://ssearch.oreilly.com/?q=",       # Website.searchUrl : 웹사이트 내 검색 주소 -> 그 자체로 절대 URL
    "article.product-result",                # Website.resultListing
    "p.title a",                             # Website.titleTag
    True,                                    # Website.absoluteUrl : 절대주소임. 
    "h1",                                    # Website.titleTag
    "section#product-description"],          # Website.bodyTag
    ["Reuters",
     "http://reuters.com",
     "http://www.reuters.com/search/news?blob=", # 상대주소
     "div.search-result-content",
     "h3.search-result-title a",
     False,
     "h1",
     'div.StandardArticleBody_body'
    ],
    ["Brookings",
     "http://www.brookings.edu",
     "https://www.brookings.edu/search/?s=",
     "div.list-content article",
     "h4.title a",
     True, 
     "h1",
     "div.post-body"]
]

# 웹사이트 객체 생성
websites = []
for site in sites:
    websites.append(Website(site[0],
                           site[1],
                           site[2],
                           site[3],
                           site[4],
                           site[5],
                           site[6],
                           site[7]))

# 검색할 주제
topics = ["python", "covid"]

# 검색
for topic in topics:
    print("{}에 대해 정보를 얻고 있습니다.".format(topic))
    for targetSite in websites:
        myCrawler.search(topic, targetSite)

python에 대해 정보를 얻고 있습니다.
접속 상태 코드 : 200
접속 상태 코드 : 200
내용을 찾았습니다 : python
URL : http://shop.oreilly.com/product/0636920028154.do
제목 : Learning Python, 5th Edition 
본문 : 
 
Get a comprehensive, in-depth introduction to the core Python language with this hands-on book. Based on author Mark Lutz’s popular training course, this updated fifth edition will help you quickly write efficient, high-quality code with Python. It’s an ideal way to begin, whether you’re new to programming or a professional developer versed in other languages. 

Complete with quizzes, exercises, and helpful illustrations,  this easy-to-follow, self-paced tutorial gets you started with both Python 2.7 and 3.3— the latest releases in the 3.X  and 2.X lines—plus all other releases in common use today. You’ll also learn some advanced language features that recently have become more common in Python code.

Explore Python’s major built-in object types such as numbers, lists, and dictionaries 
Create and process objects with

접속 상태 코드 : 200
내용을 찾았습니다 : python
URL : /article/idUSL5N26Q0CH
제목 : Monty Python fans, handkerchiefs on heads, gather to mark anniversary
본문 : 
 Monty Python fans dressed as the Gumbys gather in an attempt to set the world record for the largest gathering of people dressed as Gumbys as a part of the 50th anniversary of Monty Python's Flying Circus at the Roundhouse in London, Britain October 5, 2019. REUTERS/Simon Dawson   LONDON (Reuters) - Monty Python fans, sporting knotted handkerchiefs on their heads, rolled up trousers and Wellington boots, gathered in London on Saturday for a suitably silly celebration of the 50th anniversary of the comedy troupe. The costumes matched those of the Gumbys who were characters in the “Monty Python’s Flying Circus” series that first aired on BBC television on Oct. 5, 1969.  The Gumbys were noted for their ape-like posture, habit of speaking loudly and slowly, and the catchphrase “my brain hurts”. Dozens of Gumbys strutted outside the Roundhouse musi

접속 상태 코드 : 200
내용을 찾았습니다 : python
URL : /article/idUSS8N27T00I
제목 : Monty Python actor Terry Jones dies aged 77 - PA Media
본문 : 
 LONDON, Jan 22 (Reuters) - Terry Jones, one of the British Monty Python comedy team, has died at the age of 77, PA Media said on Wednesday. Born in Wales, Jones was also a film director and historian. He had long suffered from a rare form of dementia, FTD. Reporting by Elizabeth Howcroft and Paul Sandle; editing by
Stephen AddisonOur Standards:The Thomson Reuters Trust Principles.
접속 상태 코드 : 200
내용을 찾았습니다 : python
URL : /article/idUSKBN1YR1AP
제목 : Brussels puppet theater adds Monty Python humor to nativity tale
본문 : 
 BRUSSELS (Reuters) - A Brussels puppet theater as old as Belgium itself is staging its Christmas nativity show this year with a dash of Monty Python humor added to the traditional story of Jesus’s birth. Puppeteers rehearse Christmas nativity show at the Royal Toone Theatre in central Brussels, Belgium December 19, 2019.   REUTERS/Francois Leno

접속 상태 코드 : 200
내용을 찾았습니다 : python
URL : https://www.brookings.edu/blog/techtank/2017/11/16/leveraging-the-disruptive-power-of-artificial-intelligence-for-fairer-opportunities/
제목 : Leveraging the disruptive power of artificial intelligence for fairer opportunities
본문 : 
 
According to President Obama’s Council of Economic Advisers (CEA), approximately 3.1 million jobs will be rendered obsolete or permanently altered as a consequence of artificial intelligence technologies. Artificial intelligence (AI) will, for the foreseeable future, have a significant disruptive impact on jobs. That said, this disruption can create new opportunities if policymakers choose to harness them—including some with the potential to help address long-standing social inequities. Investing in quality training programs that deliver premium skills, such as computational analysis and cognitive thinking, provides a real opportunity to leverage AI’s disruptive power.







Makada Henry-Nickie
Fellow - Governance St

접속 상태 코드 : 200
내용을 찾았습니다 : python
URL : https://www.brookings.edu/opinions/inside-the-pentagons-secret-afghan-spy-machine/
제목 : Inside the Pentagon’s Secret Afghan Spy Machine
본문 : 
 
The Pentagon’s top researchers have rushed a classified and controversial intelligence program into Afghanistan. Known as “Nexus 7,” and previously undisclosed as a war-zone surveillance effort, it ties together everything from spy radars to fruit prices in order to glean clues about Afghan instability.

접속 상태 코드 : 200
내용을 찾았습니다 : python
URL : https://www.brookings.edu/blog/techtank/2016/03/01/idea-to-retire-old-methods-of-policy-education/
제목 : Idea to Retire: Old methods of policy education
본문 : 
 
Public policy and public affairs schools aim to train competent creators and implementers of government policy. While drawing on the principles that gird our economic and political systems to provide a well-rounded education, like law schools and business schools, policy schools provide professional training.

...

접속 상태 코드 : 200
내용을 찾았습니다 : covid
URL : https://www.brookings.edu/podcast-episode/how-might-covid-19-affect-the-global-economy/
제목 : How might COVID-19 affect the global economy?
본문 : 
 
As COVID-19 continues to spread around the world, Warwick J. McKibbin joined us from his home in Australia to discuss how the novel coronavirus may disrupt the global economy. McKibbin, a nonresident senior fellow at Brookings, authored a recent report outlining seven different scenarios of how COVID-19 might evolve and the implications each scenario would have on macroeconomic outcomes and financial markets.
As part of their conversation, McKibbin and Dollar also discuss how this epidemic compares to SARS and why it will make fulfilling the recent trade deal between China and the United States difficult.
A transcript of their discussion is available here.

Related content: 
The global macroeconomic impacts of COVID-19: Seven scenarios
What are the possible economic effects of COVID-19 on the world econ

접속 상태 코드 : 200
내용을 찾았습니다 : covid
URL : https://www.brookings.edu/research/the-global-macroeconomic-impacts-of-covid-19-seven-scenarios/
제목 : The global macroeconomic impacts of COVID-19: Seven scenarios
본문 : 
 
The outbreak of coronavirus named COVID-19 has disrupted the Chinese economy and is spreading globally. The evolution of the disease and its economic impact is highly uncertain which makes it difficult for policymakers to formulate an appropriate macroeconomic policy response. In order to better understand possible economic outcomes, this paper explores seven different scenarios of how COVID-19 might evolve in the coming year using a modelling technique developed by Lee and McKibbin (2003) and extended by McKibbin and Sidorenko (2006). It examines the impacts of different scenarios on macroeconomic outcomes and financial markets in a global hybrid DSGE/CGE general equilibrium model.







Warwick J. McKibbin
Nonresident Senior Fellow - Economic Studies Co-Director - Climate and

접속 상태 코드 : 200
내용을 찾았습니다 : covid
URL : https://www.brookings.edu/blog/education-plus-development/2020/03/11/covid-19-outbreak-highlights-critical-gaps-in-school-emergency-preparedness/
제목 : COVID-19 outbreak highlights critical gaps in school emergency preparedness
본문 : 
 
The COVID-19 epidemic sweeping the globe has affected millions of students, whose school closures have more often than not caught them, their teachers, and families by surprise. For some, it means missing class altogether, while others are trialing online learning—often facing difficulties with online connections, as well as motivational and psychosocial well-being challenges. These problems point to a critical gap in school-based contingency planning within broader education sector preparedness planning and emergency management.	






Allison Anderson
Former Brookings Expert





Contingency planning is a management tool to analyze the impact of potential crises and ensure appropriate arrangements are made to respo

접속 상태 코드 : 200
내용을 찾았습니다 : covid
URL : https://www.brookings.edu/blog/techtank/2020/03/13/the-u-s-lacks-health-information-technologies-to-stop-covid-19-epidemic/
제목 : The US lacks health information technologies to stop COVID-19 epidemic
본문 : 
 







Niam Yaraghi
Nonresident Fellow - Governance Studies, Center for Technology Innovation

 Twitter
@niamyaraghi





The COVID-19 pandemic highlights the crucial importance of health information technology and data interoperability. The pandemic has shattered our common beliefs about the type and scope of health information exchange. It has shown us that the definition of health data should no longer be limited to medical data of patients and instead should encompass a much wider variety of data types from individuals’ online and offline activity. Moreover, the pandemic has proven that healthcare is not local. In an interconnected world, with more individuals traveling long distances than ever before, it is naïve to look at regions in iso

접속 상태 코드 : 200
내용을 찾았습니다 : covid
URL : https://www.brookings.edu/blog/the-avenue/2020/03/19/covid-19-puts-americas-low-wage-workforce-in-an-even-worse-position/
제목 : COVID-19 puts America’s low-wage workforce in an even worse position
본문 : 
 







Martha Ross
Fellow - Metropolitan Policy Program

 Twitter
marthahross








Nicole Bateman
Research Analyst - Metropolitan Policy Program

 Twitter
nicbateman1





As we’ve written previously, 53 million people in the U.S.—44% of the country’s workers—earn low wages. Their median hourly earnings are $10.22, and for those who work full time year-round, median annual earnings are about $24,000. These workers already sit in precarious economic circumstances, but as the COVID-19 pandemic sweeps the globe and fears of a recession abound, they’re at greater risk than ever.
》Explore the interactive
Some of the most common low-wage jobs are inherently interpersonal in nature (think retail and food service) and are thus uniquely vulnerable to th

### 빌보드 사이트 bts 검색 결과 크롤링
- url : https://www.billboard.com/
- searchUrl : https://www.billboard.com/search/
- resultListing : div class="container search-page"
- resultUrl : result__text__title 아래 a.
- absoluteUrl : False
- 제목 태그 : div.result__text__title
- 본문 태그 : 제목 태그 아래 p
    
    
 접속은 되는데 가져오는 걸 못한다???????????????????????????????

In [44]:
# crawler 객체 생성
billboard_crawler = Crawler()

# 사이트 정보
billboard = ["BillBoard",
            "https://www.billboard.com/",
            "https://www.billboard.com/search/",
            "div.container.search-page",
            "div.result__text__title a",
            False,
            "div.result__text__title",
            "div.result__Text__title p"]

billboard_website = Website(billboard[0], 
                            billboard[1],
                            billboard[2],
                            billboard[3],
                            billboard[4],
                            billboard[5],
                            billboard[6],
                            billboard[7])

topics = ["BTS", "Beatles"]

for topic in topics:
    print("{}에 대해 검색합니다.".format(topic))
    billboard_crawler.search(topic, billboard_website)

BTS에 대해 검색합니다.
접속 상태 코드 : 200
접속 상태 코드 : 404
Beatles에 대해 검색합니다.
접속 상태 코드 : 200
접속 상태 코드 : 404


### 네이버뮤직 bts 검색 결과 크롤링
- url : https://music.naver.com/
- searchUrl : https://music.naver.com/search/search.nhn?query=
- resultListing : div.section
- resultUrl : td.name a href
- absoluteUrl : True
- 아티스트 태그 : h3
- 가사 태그 : ul.lst_detail5 li p


In [51]:
naverMusic_crawler = Crawler()

# 사이트 정보
naverMusic = ["NaverMusic",
            "https://music.naver.com/",
            "https://music.naver.com/search/search.nhn?query=",
            "div.section",
            "td.name a",
            True,
            "h3",
            "ul.lst_detail5 li p"]

naverMusic_website = Website(naverMusic[0], 
                            naverMusic[1],
                            naverMusic[2],
                            naverMusic[3],
                            naverMusic[4],
                            naverMusic[5],
                            naverMusic[6],
                            naverMusic[7])

naverMusic_crawler.search("방탄소년단", naverMusic_website)

접속 상태 코드 : 200


IndexError: list index out of range

## 2.2. 링크를 통한 사이트 크롤링

 특정 URL 패턴과 일치하는 링크를 모두 따라갈 수 있는, 유연한 크롤러를 만든다.
 
 * 활용
     - 사이트 전체에서 데이터를 수집해야 할 때
     - 사이트의 페이지가 적절히 구조화되지 않았을 때
     - 사이트 페이지가 광범위하게 분산되었을 때

  
1. `Content` 클래스 : 위와 동일.

2. `Website` 클래스
    * 크롤러가 찾아야 할 링크의 위치에 대한 구체적인 지침이 없으므로, **어떤 종류의 페이지를 선택할지** 지정하는 규칙이 필요함.(*검색 페이지가 절대URL인지 상대URL인지는 중요하지 않음*)
    * `targetPattern`(대상 URL에 대한 정규표현식)과 `absoluteURL`(대상 URL이 절대주소인지 나타내는 불리언 변수)를 사용하여 어떤 종류의 페이지를 지정할 지 규칙을 정한다.

3. `Crawler` 클래스
    * 각 사이트의 홈페이지에서 시작해 내부 링크를 찾는다.
    * 발견된 각 내부 링크의 내용에 대해 구문을 분석한다.
    
* 작동 원리 특징
    - `reuters` 변수 자체가 `Website` 객체. 위에서는 웹사이트 목록 리스트로 만들었고, `for문`을 통해 리스트를 순회하며 Website 객체를 만들었다. 따라서, 크롤러 하나가 웹사이트 리스트를 순회하지는 못한다. 각 웹사이트마다 새 크롤러를 `Website` 객체화하여 인스턴스화한다.
    - 페이지 하나에서 모두 다 정보를 가져온 뒤라면, 그 페이지 크롤링이 완료된 후 더 이상 크롤링을 계속하지 않는다. 여러 가지 링크 유형을 정해 놓고, 다른 링크도 따라가면서 대상 패턴이 포함된 URL을 찾도록 할 수 있다.
    
    
    
### 크롤러에 웹사이트 자체 정보를 저장할것인지, 아니면 리스트를 순회하며 웹사이트 정보를 가져올지는 스스로 판단하여 결정해야 한다.
   

In [2]:
# module import
import re
import requests
from bs4 import BeautifulSoup

# Website 클래스 : 웹사이트 구조.
class Website:
    def __init__(self, name, url, targetPattern, absoluteUrl, titleTag, bodyTag):
        self.name = name
        self.url = url
        self.targetPattern = targetPattern
        self.absoluteUrl = absoluteUrl
        self.titleTag = titleTag
        self.bodyTag = bodyTag
        
# Content 클래스: 내용에 맞게 구조화.
class Content:
    def __init__(self, url, title, body):
        self.url = url
        self.title = title
        self.body = body
        
    def print(self):
        print("URL : {}".format(self.url))
        print("제목 : {}".format(self.title))
        print("본문 : \n{}".format(self.body))

# Crawler 클래스
class Crawler:
    def __init__(self, site):
        self.site = site
        self.visited = []

    # 페이지 접속 기능
    def getPage(self, url):
        session = requests.Session()
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
               'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}
        try:
            req = session.get(url, headers = headers)
            print("접속 상태 코드 : {}".format(req.status_code)) # 접속 상태 확인하기 위한 코드.
        except requests.exceptions.RequestException as e:
            print("에러 : {}".format(e))
            return None
        bs = BeautifulSoup(req.text, 'html.parser')
        return bs
    
    def safeGet(self, pageObj, selector):
        selectedElems = pageObj.select(selector)
        if selectedElems is not None and len(selectedElems) > 0 :
            return "\n".join([elem.get_text() for elem in selectedElems])
        return ''
    
    def parse(self, url):
        bs = self.getPage(url)
        if bs is not None:
            title = self.safeGet(bs, self.site.titleTag)
            body = self.safeGet(bs, self.site.bodyTag)
            if title != "" and body != "":
                content = Content(url, title, body)
                content.print()
    
    # 사이트 홈페이지에서 내부 페이지 링크를 가져온다.
    def crawl(self):
        bs = self.getPage(self.site.url)
        targetPages = bs.findAll('a', href = re.compile(self.site.targetPattern))
        for targetPage in targetPages:
            targetPage = targetPage.attrs["href"]
            if targetPage not in self.visited: # 방문하지 않은 페이지
                self.visited.append(targetPage)
                if not self.site.absoluteUrl: # 절대주소 아니라면
                    targetPage = '{}{}'.format(self.site.url, targetPage)
                self.parse(targetPage)

In [3]:
# reuter Website 객체                
reuters = Website('Reuters',
                     'https://www.reuters.com',
                     '^(/article/)', # 정규표현식 : /article/로 시작해서 기사 제목이 온다.
                     False, # 절대URL 아님. = 상대URL이다.
                     'h1',
                     'div.StandardArticleBody_body') # 본문 컨텐츠 영역.

                  
reuter_crawler = Crawler(reuters)
reuter_crawler.crawl()

접속 상태 코드 : 200
접속 상태 코드 : 200
URL : https://www.reuters.com/article/us-health-coronavirus-usa-adams/u-s-states-cities-desperate-for-coronavirus-help-military-prepares-idUSKBN21A27M
제목 : U.S. states, cities desperate for coronavirus help, military prepares
본문 : 
NEW YORK/WASHINGTON (Reuters) - Several more U.S. governors on Monday joined the procession of states ordering millions of Americans to stay at home to slow the spread of the coronavirus, while President Trump signaled he’s considering a move in the opposite direction. Public health authorities have pushed for the stay-at-home restrictions as essential to curb widespread transmission of a highly contagious respiratory virus that has infected more than 42,000 people in the United States, killing at least 559.  While a wave of statewide social distancing measures expanded, further stifling the U.S. economy amid another day of plunging stock prices and growing fears of a global recession, Trump said, “we cannot let the cure be wors

접속 상태 코드 : 200
URL : https://www.reuters.com/article/us-health-coronavirus-who-hack-exclusive/exclusive-elite-hackers-target-who-as-coronavirus-cyberattacks-spike-idUSKBN21A3BN
제목 : Exclusive: Elite hackers target WHO as coronavirus cyberattacks spike
본문 : 
WASHINGTON/LONDON (Reuters) - Elite hackers tried to break into the World Health Organization earlier this month, sources told Reuters, part of what a senior agency official said was a more than two-fold increase in cyberattacks. Traffic passes the Regional Office for the Americas of the World Health Organization (WHO) during the coronavirus disease (COVID-19) outbreak in Washington, D.C., U.S. March 22, 2020. REUTERS/Raphael SatterWHO Chief Information Security Officer Flavio Aggio said the identity of the hackers was unclear and the effort was unsuccessful. But he warned that hacking attempts against the agency and its partners have soared as they battle to contain the coronavirus, which has killed more than 15,000 worldwide. The 

접속 상태 코드 : 200
URL : https://www.reuters.com/article/us-health-coronavirus-usa-congress/senate-talks-to-resume-hopes-high-massive-coronavirus-bill-will-pass-idUSKBN21B18D
제목 : Senate talks to resume, hopes high massive coronavirus bill will pass
본문 : 
WASHINGTON (Reuters) - The U.S. Senate could pass a $2 trillion coronavirus economic stimulus package as soon as Tuesday, negotiators said, insisting they had made significant progress despite failing so far to reach a bipartisan deal on the sweeping legislation. FILE PHOTO: White House Legislative Affairs Director Eric Ueland and U.S. Secretary of the Treasury Steven Mnuchin walk to a meeting during negotiations on a coronavirus disease (COVID-19) relief package on Capitol in Washington, U.S., March 23, 2020.  REUTERS/Joshua RobertsSteven Mnuchin, President Donald Trump’s treasury secretary, said he would return to the Capitol for more talks on Tuesday, after a day of negotiations ended at midnight without an agreement. Neither Mnuchin n

접속 상태 코드 : 200
URL : https://www.reuters.com/article/us-health-coronavirus-banks-insight/banks-struggle-to-ride-to-the-rescue-in-europes-cash-crunch-battle-idUSKBN21B0OE
제목 : Banks struggle to ride to the rescue in Europe's cash crunch battle
본문 : 
MILAN/MADRID/FRANKFURT (Reuters) - Corrado Sforza Fogliani is on the frontlines of European efforts to keep the region’s economy alive amid the coronavirus pandemic. FILE PHOTO: The financial district is photographed on early evening in Frankfurt, Germany, January 29, 2019.  REUTERS/Kai PfaffenbachHis small cooperative bank, Banca di Piacenza, in Italy’s virus-ravaged north, has been flooded with applications from customers seeking loan extensions, debt relief and renewal of credit lines. Buried in paperwork and with Rome and banking lobbies still at odds over who should be on the hook for defaults when a six-month debt holiday ends, Banca di Piacenza’s loan officers have only been able to process a fraction of the 1,000 applications they ha

접속 상태 코드 : 200
URL : https://www.reuters.com/article/us-health-coronavirus-iran/about-half-of-irans-state-workers-stay-at-home-as-coronavirus-death-toll-nears-2000-idUSKBN21B0SR?il=0
제목 : About half of Iran's state workers are staying home: president
본문 : 
FILE PHOTO: Iranan President Hassan Rouhani speaks during a meeting of the Iranian government task force on the coronavirus, in Tehran, Iran March 21, 2020. Official Presidential website/Handout via REUTERS  DUBAI (Reuters) - Iran’s President Hassan Rouhani said on Tuesday about half of all government employees were staying at home as part of measures to contain the coronavirus outbreak, state TV reported. Iran is one of the hardest hit countries outside China, with more than 23,000 confirmed infections and more than 1,800 dead, according to the latest figures issued on Monday. Another measure to contain the outbreak, the temporary release of prisoners, will be extended until the end of the current Iranian month of Farvardin, about A

...

접속 상태 코드 : 200
URL : https://www.reuters.com/article/us-health-coronavirus-carmakers-tesla/tesla-ceo-says-bought-ventilators-in-china-for-u-s-idUSKBN21B19Q
제목 : Tesla CEO says bought ventilators in China for U.S
본문 : 
FILE PHOTO: SpaceX owner and Tesla CEO Elon Musk arrives on the red carpet for the automobile awards "Das Goldene Lenkrad" given by a German newspaper in Berlin, Germany, November 12, 2019. REUTERS/Hannibal Hanschke/File PhotoFRANKFURT (Reuters) - Tesla bought hospital ventilators in China and shipped them to the United States, CEO Elon Musk said on Tuesday. Tesla’s purchase comes as governments across the globe appeal to automakers and aerospace companies help procure or make ventilators and other medical equipment. “China had an oversupply, so we bought 1255 FDA-approved ResMed, Philips & Medtronic ventilators on Friday night & airshipped them to LA,” Musk said on Twitter. California Governor Gavin Newsom on Tuesday said the state would receive 1,000 ventilators from Te

접속 상태 코드 : 200
URL : https://www.reuters.com/article/us-health-coronavirus-iran/about-half-of-irans-state-workers-stay-at-home-as-coronavirus-death-toll-nears-2000-idUSKBN21B0SR
제목 : About half of Iran's state workers stay at home as coronavirus death toll nears 2,000
본문 : 
FILE PHOTO: Iranan President Hassan Rouhani speaks during a meeting of the Iranian government task force on the coronavirus, in Tehran, Iran March 21, 2020. Official Presidential website/Handout via REUTERS  DUBAI (Reuters) - Iran’s President Hassan Rouhani said on Tuesday about half of all government employees were staying at home as part of measures to contain the coronavirus outbreak, state TV reported. Another measure to contain the outbreak, the temporary release of prisoners, would be extended until the end of the current Iranian month of Farvardin, around April 18, Rouhani said.  Iran is one of the countries hit hardest by the pandemic outside China.  The death toll from the disease in Iran increased by 122 t

접속 상태 코드 : 200
URL : https://www.reuters.com/article/us-revolut-app/british-fintech-revolut-launches-banking-app-in-u-s-idUSKBN21B1AR
제목 : British fintech Revolut launches banking app in U.S.
본문 : 
FILE PHOTO: A Revolut logo is seen in this illustration taken January 6, 2020. REUTERS/Dado Ruvic/Illustration(Reuters) - British fintech firm Revolut Ltd on Tuesday launched its banking app and debit cards for customers in the United States as it seeks to expand beyond its base in the UK and Europe. The rapidly growing app, started in 2015, has added 10 million users and raised $836 million in funding till date. Revolut said its app will enable users to deposit their salary, keep a track of their spending and set monthly budgets for travel and groceries.   The company also said it intends to offer commission-free stock trading and cryptocurrency trading in the near future, making it potential competition for U.S. fintech firms such as Robinhood. Revolut raised here $500 million in February 

접속 상태 코드 : 200
URL : https://www.reuters.com/article/us-health-coronavirus-usa-congress/mnuchin-hopes-deal-is-very-close-on-2-trillion-coronavirus-aid-package-in-u-s-senate-idUSKBN21A1O0
제목 : Mnuchin hopes deal is 'very close' on $2-trillion coronavirus aid package in U.S. Senate
본문 : 
WASHINGTON (Reuters) - The U.S. Treasury secretary and the Senate Democratic leader voiced confidence late on Monday for a deal to be reached soon on a far-reaching coronavirus economic stimulus package that had been stalled in the U.S. Senate as lawmakers haggled over it. Negotiators made great progress on the bipartisan, $2-trillion stimulus measure on Monday, but without striking a final pact as they had hoped, Treasury Secretary Steven Mnuchin and Senate Democratic Leader Chuck Schumer told reporters. “There are still a couple of open issues,” Mnuchin said, just before midnight. “I think we are very hopeful that this can be closed out tomorrow.” Schumer said he thought the Senate could vote as soon a

접속 상태 코드 : 200
URL : https://www.reuters.com/article/us-reuters-editorial-leadership/reuters-editorial-leadership-idUSKBN1KE2SD
제목 : Reuters Leadership
본문 : 
Michael FriedenbergPresident Stephen J. AdlerEditor-in-Chief Josh LondonChief Marketing Officer Michael YoungChief Technology Officer Editorial Leadership: Reg ChuaGlobal Managing Editor, Operations Alessandra GalloniGlobal Managing Editor, News Planning and Creation Simon RobinsonGlobal Managing Editor, Newsroom Tiffany WuGlobal Managing Editor, Editing, Curation and Publishing Alix FreedmanGlobal Editor, Ethics and Standards John PullmanGlobal Head of Visuals Michael WilliamsGlobal Enterprise Editor Jane BarrettEditor, Media News Strategy Rob CoxEditor, Breakingviews Jonathan LeffEditor, Financial News Strategy Dan FlynnRegional Editor, Latin America Kevin KrolickiRegional Editor, Asia Samia NakhoulRegional Editor, Middle East Alex ZavisRegional Editor, Sub-Saharan Africa Mark BendeichRegional Editor, Europe Kieran MurrayRegiona

## 2.3. 여러 페이지 유형 크롤링

 2.2에서처럼 한 웹사이트 내에 존재하는 내부 링크를 모두 식별하고 크롤링하면, 무엇을 얻게될 지 정확히 알 수 없다. 위만 하더라도 기사가 엄청 많이 나오고, 심지어 밑에 Editorial까지 나온다. 따라서 **페이지 유형**을 식별하는 방법이 있다.

--- 

* URL에 따른 식별 방법
    - 예를 들어, 블로그 게시물이면 http:\~/blog/로 시작할 수 있다.

* 특정 필드의 존재 여부에 따른 식별 방법
    - 예를 들어, 보도자료는 페이지에 *날짜*가 있지만 *작성자 이름*이 없을 수 있다.
    - 제품 페이지의 경우, *제목, 주 이미지, 가격*은 있지만, *주요 콘텐츠*가 없을 수 있다.

* 페이지를 식별할 수 있는 특정 태그의 존재 여부에 따른 식별 방법
    - 예를 들어, 크롤러를 사용하기 전, 애초에 페이지 구조 자체에 `<div id = "relatedproducts">`와 같은 요소가 있다면, 제품 페이지로 판단할 수 있다.
    
---

 위와 같은 여러 유형의 페이지 객체를 모두 파이썬으로 만들고, 크롤링을 진행할 수 있다. 다음의 두 가지 방법이 대표적이다.

1. `pageType` 속성 추가
    - 위에서 계속 다뤘던 `Website` 객체에 `pageType` 속성을 추가한다.
    - 이런 페이지를 SQL 등 데이터베이스에 저장하고, 패턴 유형을 보고 모든 페이지가 동일한 테이블에 저장되고 `pageType` 열이 추가된다는 것만 확인하면 된다.

2. 하위 클래스 사용
    - 서로 다른 유형의 페이지나 컨텐츠라면, 하위 클래스를 사용할 수 있다.

    

### 1. `pageType` 속성 추가

In [ ]:
class Website:
    def __init__(self, pageType, name, url, searchUrl, resultListing, resultUrl, absoluteUrl, titleTag, bodyTag):
        self.name = name
        self.url = url
        self.titleTag = titleTag
        self.bodyTag = bodyTag
        self.pageType = pageType

### 2. 페이지 유형 객체에서 참조할 하위 클래스 생성

* `Product` 클래스 : `Website` 클래스 확장. 제품에만 적용되는 `productNum`, `price` 속성 추가.
* `Article` 클래스 : `Product` 클래스에 적용되지 않는 `body`, `date` 속성 추가.


In [ ]:
class Webpage:
    def __init__(self, name, url, titleTag):
        self.name = name
        self.url = url
        self.titleTag = titleTag

# 아래 클래스를 하위 페이지 유형에서 참고한다.
class Product(Website) : # 제품 페이지 스크랩에 필요한 정보
    def __init__(self, name, url, titleTag, productNum, price):
        Website.__init__(self, name, url, TitleTag)
        self.productNumTag = productNumTag
        self.proceTag = priceTag

class Article(Website): # 기사 페이지 스크랩에 필요한 정보
    def __init__(self, name, url, titleTag, bodyTag, dateTag):
        Website.__init__(self, name, url, titleTag)
        self.bodyTag = bodyTag
        self.dateTag = dateTag
        